In [2]:
import numpy as np
import faiss
import eccv_caption
from omegaconf import OmegaConf
from lavis.models import load_preprocess
from lavis.models.albef_models.albef_retrieval import AlbefRetrieval
import clip
import torch
from PIL import Image
import json
import os
import time
from tqdm.auto import tqdm


/Users/weedcuper/miniconda3/envs/lavis310/lib/python3.10/site-packages/eccv_caption/metrics.py:21: UserWarning: failed to import `ujson`. use `json` instead.
  warnings.warn('failed to import `ujson`. use `json` instead.')
/Users/weedcuper/miniconda3/envs/lavis310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/weedcuper/miniconda3/envs/lavis310/lib/python3.10/site-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/Users/weedcuper/miniconda3/envs/lavis310/lib/python3.10/site-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please us

In [3]:
DATASET_PATH = "dataset/coco2014"
ANN_PATH = f"{DATASET_PATH}/annotations/karpathy_test.json"
DATA_DIR = f"{DATASET_PATH}/eccv/data"
MODEL_NAME = "ViT-B/32"
IMAGE_BATCH = 64
TEXT_BATCH = 256
RETRIEVE_K = 50
ALBEF_IMAGE_BATCH = 32
ALBEF_CFG_PATH = "albef_retrieval_base.yaml"

In [4]:
def load_karpathy_test(ann_path, coco_root):
    data = json.load(open(ann_path, "r", encoding="utf-8"))
    images = []
    image_ids = []
    captions = []
    caption_ids = []

    for img in data["images"]:
        if img["split"] != "test":
            continue
        img_path = os.path.join(coco_root, img["filepath"], img["filename"])
        images.append(img_path)
        image_ids.append(img["cocoid"])
        for sent in img["sentences"]:
            captions.append(sent["raw"].strip())
            caption_ids.append(sent["sentid"])

    return images, image_ids, captions, caption_ids

In [5]:
def encode_images(model, preprocess, image_paths, batch_size, device):
    feats = []
    total = len(image_paths)
    for i in range(0, total, batch_size):
        batch_paths = image_paths[i:i + batch_size]
        images = [preprocess(Image.open(p).convert("RGB"))
                  for p in batch_paths]
        image_input = torch.stack(images).to(device)
        with torch.no_grad():
            batch_feats = model.encode_image(image_input)
        feats.append(batch_feats)
        # if (i // batch_size) % 10 == 0:
        #     done = min(i + batch_size, total)
        #     print(f"Encoded images: {done}/{total}")
    feats = torch.cat(feats, dim=0)
    return feats / feats.norm(dim=1, keepdim=True)

In [6]:
def encode_texts(model, texts, batch_size, device):
    feats = []
    total = len(texts)
    for i in range(0, total, batch_size):
        batch = texts[i:i + batch_size]
        tokens = clip.tokenize(batch).to(device)
        with torch.no_grad():
            batch_feats = model.encode_text(tokens)
        feats.append(batch_feats)
        # if (i // batch_size) % 10 == 0:
        #     done = min(i + batch_size, total)
        #     print(f"Encoded captions: {done}/{total}")
    feats = torch.cat(feats, dim=0)
    return feats / feats.norm(dim=1, keepdim=True)

In [7]:
def topk_faiss(text_feats, img_feats, k):
    txf = text_feats.detach().cpu().numpy().astype("float32", copy=False)
    imf = img_feats.detach().cpu().numpy().astype("float32", copy=False)
    txf = np.ascontiguousarray(txf)
    imf = np.ascontiguousarray(imf)
    k = min(k, imf.shape[0])
    index = faiss.IndexFlatIP(imf.shape[1])
    index.add(imf)
    _, indices = index.search(txf, k)
    return indices

In [8]:
def albef_rerank(albef_model, albef_preprocess, captions, image_paths, t2i_rank, batch_size, device):
    image_feats = []
    total = len(image_paths)
    batch_start = time.time()
    for i in tqdm(range(0, total, batch_size), desc="ALBEF image feats"):
        batch_paths = image_paths[i:i + batch_size]
        images = [albef_preprocess(Image.open(p).convert("RGB"))
                  for p in batch_paths]
        image_input = torch.stack(images).to(device)
        with torch.no_grad():
            batch_feats = albef_model.visual_encoder.forward_features(
                image_input)
        image_feats.append(batch_feats.cpu())
    image_feats = torch.cat(image_feats, dim=0)
    batch_time = time.time() - batch_start

    print(f"ALBEF image feats: {total} in {batch_time:.3f}s")
    
    reranked = []
    total_caps = len(captions)
    batch_start = time.time()
    for i in tqdm(range(total_caps), desc="ALBEF rerank"):
        cand = t2i_rank[i]
        img_feat = image_feats[cand].to(device)
        encoder_att = torch.ones(
            img_feat.size()[:-1], dtype=torch.long).to(device)
        text_input = albef_model.tokenizer(
            [captions[i]] * len(cand),
            padding="max_length",
            truncation=True,
            max_length=35,
            return_tensors="pt",
        ).to(device)
        with torch.no_grad():
            output = albef_model.text_encoder(
                text_input.input_ids,
                attention_mask=text_input.attention_mask,
                encoder_hidden_states=img_feat,
                encoder_attention_mask=encoder_att,
                return_dict=True,
            )
            scores = albef_model.itm_head(
                output.last_hidden_state[:, 0, :])[:, 1]
        order = torch.argsort(scores, descending=True)
        reranked.append([cand[j] for j in order.tolist()])

    batch_time = time.time() - batch_start
    print(f"ALBEF rerank: {total_caps} in {batch_time:.3f}s")
    return reranked

In [9]:
device = "cuda" if torch.cuda.is_available(
) else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

images, image_ids, captions, caption_ids = load_karpathy_test(
    ANN_PATH, DATASET_PATH)
print(f"Test images: {len(images)}")
print(f"Test captions: {len(captions)}")
print(f"Test captions: {len(caption_ids)}")

model, preprocess = clip.load(MODEL_NAME, device=device)
model.eval()

t0 = time.time()
image_feats = encode_images(model, preprocess, images, IMAGE_BATCH, device)
text_feats = encode_texts(model, captions, TEXT_BATCH, device)
print(f"Encoded in {time.time() - t0:.1f}s")

i2t_rank = topk_faiss(image_feats, text_feats, RETRIEVE_K).tolist()
t2i_rank = topk_faiss(text_feats, image_feats, RETRIEVE_K).tolist()

Using device: mps
Test images: 5000
Test captions: 25010
Test captions: 25010
Encoded in 61.7s


In [10]:
np.shape(t2i_rank)

(25010, 50)

In [42]:
albef_cfg = OmegaConf.load(ALBEF_CFG_PATH)
albef_model = AlbefRetrieval.from_config(albef_cfg.model)
albef_model.eval()
albef_model = albef_model.to(device)
albef_vis, _ = load_preprocess(albef_cfg.preprocess)

t2i_rank = albef_rerank(
    albef_model, albef_vis["eval"], captions, images, t2i_rank, ALBEF_IMAGE_BATCH, device
)

/Users/weedcuper/miniconda3/envs/lavis310/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


reshape position embedding from 256 to 576
reshape position embedding from 256 to 576


ALBEF image feats: 100%|██████████| 157/157 [02:53<00:00,  1.11s/it]


ALBEF image feats: 5000 in 193.383s


ALBEF rerank: 100%|██████████| 25010/25010 [1:37:09<00:00,  4.29it/s]


ALBEF rerank: 25010 in 5829.100s


In [11]:
i2t = {
    image_ids[i]: [caption_ids[j] for j in i2t_rank[i]]
    for i in range(len(image_ids))
}
t2i = {
    caption_ids[i]: [image_ids[j] for j in t2i_rank[i]]
    for i in range(len(caption_ids))
}

In [12]:

metric = eccv_caption.Metrics()
scores = metric.compute_all_metrics(
    i2t_retrieved_items=i2t,
    t2i_retrieved_items=t2i,
    target_metrics=["eccv_map_at_r", "eccv_rprecision", "eccv_r1"],
    Ks=[1],
    verbose=True,
)

print("ECCV T2I metrics:")
print(f"Map@R: {scores['eccv_map_at_r']['t2i']:.2f}")
print(f"R-P: {scores['eccv_rprecision']['t2i']:.2f}")
print(f"R@1: {scores['eccv_r1']['t2i']:.2f}")

100%|██████████| 1332/1332 [00:00<00:00, 128134.97it/s]

ECCV T2I metrics:
Map@R: 0.31
R-P: 0.41
R@1: 0.68


In [ ]:
import json
OUTPUT_PATH = "dataset/coco2014/annotations/karpathy_paraphrases.json"
with open(OUTPUT_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

data[0]

5000